# Assignment 3

Author: [Lucas David](http://github.com/lucasdavid)  
This notebook can be downloaded at https://github.com/lucasdavid/mo850/

In [1]:
import os
from itertools import combinations

import numpy as np
import pandas as pd
import scipy
from scipy import stats
import scikit_posthocs as sp
import statsmodels.stats.contingency_tables
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import multipletests

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from keras import Input, Model, callbacks
from keras.layers import Dense, BatchNormalization, Activation
from keras.datasets.cifar10 import load_data

import matplotlib
from matplotlib import pyplot

%matplotlib inline

Using TensorFlow backend.


## Unpaired Data

In [2]:
data_dir = '../data/3/'

datasets = [letter + '.csv' for letter in 'abcde']
print('The following datasets will be loaded:', *datasets)

datasets = [pd.read_csv(os.path.join(data_dir, d), header=None, names=['measure'])
            for d in datasets]

for tag, d in zip('abcde', datasets):
    d['group'] = tag

print('Sample of a loaded dataset:',
      datasets[0].head(),
      sep='\n')

The following datasets will be loaded: a.csv b.csv c.csv d.csv e.csv
Sample of a loaded dataset:
    measure group
0  4.249030     a
1  5.542826     a
2  5.161981     a
3  2.267553     a
4  4.155343     a


In [3]:
s, p = stats.f_oneway(*(d['measure'] for d in datasets))
print('p-value for anova test:', p)

p-value for anova test: 3.3595478270572274e-12


In [4]:
merged_datasets = pd.concat(datasets)
r = pairwise_tukeyhsd(merged_datasets['measure'],
                      groups=merged_datasets['group'])
print(r)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  a      b    -0.5373   -1.514 0.4395 False 
  a      c     1.831    0.8718 2.7901  True 
  a      d     1.908    1.0037 2.8123  True 
  a      e     1.7006   0.7414 2.6597  True 
  b      c     2.3682   1.3405 3.396   True 
  b      d     2.4453   1.4685 3.422   True 
  b      e     2.2379   1.2101 3.2656  True 
  c      d     0.077   -0.8821 1.0362 False 
  c      e    -0.1304  -1.1414 0.8807 False 
  d      e    -0.2074  -1.1665 0.7518 False 
--------------------------------------------


In [5]:
s, p = stats.kruskal(*(d['measure'] for d in datasets))
print('p-value for Kruskal-Wallis:', p)

p-value for Kruskal-Wallis: 2.275373540934311e-09


In [6]:
pairs_indices = list(combinations('abcde', 2))
pairs_data = combinations([d['measure'] for d in datasets], 2)

ps = [stats.ranksums(x, y)[1] for x, y in pairs_data]
ps_holm = multipletests(ps, method='holm')[1]
ps_bonferroni = multipletests(ps, method='bonferroni')[1]

ps = pd.DataFrame({
    'pairs': pairs_indices,
    'p': ps,
    'p holm': ps_holm,
    'p bonferroni': ps_bonferroni
}).set_index('pairs')[['p', 'p holm', 'p bonferroni']]

print(ps)

                   p    p holm  p bonferroni
pairs                                       
(a, b)  2.433450e-01  0.973380      1.000000
(a, c)  1.332963e-04  0.000800      0.001333
(a, d)  5.508804e-06  0.000050      0.000055
(a, e)  1.515892e-04  0.000800      0.001516
(b, c)  6.598469e-06  0.000053      0.000066
(b, d)  9.583666e-07  0.000010      0.000010
(b, e)  2.789325e-05  0.000195      0.000279
(c, d)  5.452595e-01  1.000000      1.000000
(c, e)  8.505281e-01  1.000000      1.000000
(d, e)  5.883647e-01  1.000000      1.000000


## Paired Data

In [7]:
dataset = 'multi.csv'
print('The following datasets will be loaded:', dataset)

dataset = pd.read_csv(os.path.join(data_dir, dataset), header=None)
d_stats = pd.DataFrame({
    'mean': dataset.mean(axis=0),
    'std': dataset.std(axis=0)
})

print(dataset.head())
print(d_stats)

maybe_faster = d_stats.loc[0, 'mean'] < d_stats['mean'].loc[1:]

The following datasets will be loaded: multi.csv
           0          1          2           3           4
0  34.381581  38.230745  52.236630   59.027814   28.288420
1  78.475309  74.647168  82.374582   95.970556   28.983943
2   5.676301   8.919621  16.492051   28.646045   29.585645
3  90.357392  90.869337  98.987833  112.835174  118.070135
4  72.198253  71.068576  79.561990   92.640627   72.266390
        mean        std
0  44.040644  26.147748
1  44.989482  26.359810
2  55.163707  25.936504
3  65.438923  27.269594
4  48.367785  29.785974


In [8]:
s, p = stats.friedmanchisquare(*(dataset[c] for c in dataset.columns))
print('p-value:', p)

p-value: 9.195616602651903e-09


In [9]:
ps = [stats.wilcoxon(dataset[0], dataset[c])[1] for c in range(1, 5)]
print('p-values with wilconxon sign rank test:', *ps, sep='\n', end='\n\n')

for method in ('holm', 'bonferroni'):
    print('Using', method, 'correction:')
    rejected_hypotesis, corrected_ps, *_ = multipletests(ps, method=method)

    d = pd.DataFrame({
        'algorithm': list(range(1, 5)),
        'p': corrected_ps,
        'significantly_faster': rejected_hypotesis & maybe_faster
    }).set_index('algorithm')
    print(d, end='\n\n')

p-values with wilconxon sign rank test:
0.30878760631195334
0.0005991194011680517
0.0002930525201924893
0.12392922503869737

Using holm correction:
                  p  significantly_faster
algorithm                                
1          0.308788                 False
2          0.001797                  True
3          0.001172                  True
4          0.247858                 False

Using bonferroni correction:
                  p  significantly_faster
algorithm                                
1          1.000000                 False
2          0.002396                  True
3          0.001172                  True
4          0.495717                 False



In [10]:
print('Nemenyi procedure:')
# scikit-posthocs can be found at:
# https://github.com/maximtrp/scikit-posthocs
sp.posthoc_nemenyi_friedman(dataset.values)

Nemenyi procedure:


,0,1,2,3,4
0,-1.000000,0.900000,0.004719,0.001000,0.052271
1,0.900000,-1.000000,0.028130,0.001000,0.191307
2,0.004719,0.028130,-1.000000,0.152054,0.900000
3,0.001000,0.001000,0.152054,-1.000000,0.020215
4,0.052271,0.191307,0.900000,0.020215,-1.000000


## Extra Homework for ML Students

In [11]:
samples_used = 1000

(x, y), _ = load_data()
x = x.astype(float)
x /= 127.
x -= 1.

# Gray-scale, 1-rank tensors.
x = x.mean(axis=-1).reshape(x.shape[0], -1)
y = y.ravel()

p = np.random.permutation(x.shape[0])[:samples_used]
x, y = x[p], y[p]

print(x.shape, x[:3])
print(y.shape, y[:3])

print(' data statistics:', x.mean(), x.std())
print('label statistics:', dict(zip(*np.unique(y, return_counts=True))))

(1000, 1024) [[ 0.70603675  0.69816273  0.70341207 ...  0.42519685  0.42782152
   0.45144357]
 [-0.39370079 -0.69816273 -0.59580052 ... -0.7664042  -0.77165354
  -0.75328084]
 [ 0.68766404  0.50131234  0.49606299 ...  0.03149606  0.05249344
   0.11023622]]
(1000,) [0 8 7]
 data statistics: -0.033318702837926505 0.4933701274820272
label statistics: {0: 97, 1: 105, 2: 101, 3: 103, 4: 97, 5: 98, 6: 101, 7: 108, 8: 87, 9: 103}


In [12]:
def build_model(estimator):
    if estimator == 'a':
        return SVC()
    elif estimator == 'b':
        x = Input(shape=[1024])
        y = Dense(512, use_bias=False)(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Dense(512, use_bias=False)(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Dense(10, activation='softmax')(y)
        model = Model(inputs=x, outputs=y)
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    else:
        raise ValueError('unknown estimator of type `%s`' % estimator)

In [13]:
r = np.random.RandomState(42)

experiments = 5
answers, scores = [], []

for experiment in range(experiments):
    print('Executing experiment', experiment)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r)
    
    for train_indices, test_indices in skf.split(x, y):
        x_train, y_train = (d[train_indices] for d in (x, y))
        x_test, y_test = (d[test_indices] for d in (x, y))
        
        fit_params = {
            'a': dict(),
            'b': dict(epochs=100, verbose=0, batch_size=None,
                      validation_split=0.3,
                      callbacks=[
                          callbacks.TerminateOnNaN(),
                          callbacks.EarlyStopping(patience=5)])
        }

        for estimator in 'ab':
            e = build_model(estimator)
            e.fit(x_train, y_train, **fit_params[estimator])
            
            if isinstance(e, Model):
                evaluation = e.evaluate(x_test, y_test, verbose=0, batch_size=None)
                score = evaluation[1]
                scores += [score]
            elif isinstance(e, SVC):
                scores += [e.score(x_test, y_test)]
            
            p = e.predict(x_test)
            
            if isinstance(e, Model):
                p = np.argmax(p, axis=-1)
            
            answers += [p.astype(int)]

Executing experiment 0
Executing experiment 1
Executing experiment 2
Executing experiment 3
Executing experiment 4


In [14]:
answers, scores = map(np.asarray, (answers, scores))
scores_a = scores[::2]
scores_b = scores[1::2]

print('model a\'s average score:', scores_a.mean())
print('model b\'s average score:', scores_b.mean())

print('On average, model',
      'a' if scores_a.mean() > scores_b.mean() else 'b',
      'performed better.')

model a's average score: 0.2520001280081925
model b's average score: 0.2202348950805874
On average, model a performed better.


In [15]:
s, p = stats.wilcoxon(scores_a, scores_b)
print('p-value from wilconxon test:', p)

p-value from wilconxon test: 0.007685794055213263


/home/ldavid/envs/tf/lib/python3.6/site-packages/scipy/stats/morestats.py:2385: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


In [16]:
answers_a = np.concatenate(answers[::2])
answers_b = np.concatenate(answers[1::2])

print('samples of model a\'s predictions:', answers_a)
print('samples of model b\'s predictions:', answers_b)

rc_table = pd.crosstab(answers_a, answers_b)
print('contigency table:', rc_table, sep='\n')

p = statsmodels.stats.contingency_tables.mcnemar(rc_table).pvalue
print('p-value from mcnemar test:', p)

samples of model a's predictions: [9 7 9 ... 1 3 7]
samples of model b's predictions: [9 5 8 ... 3 3 6]
contigency table:
col_0    0    1   2    3    4    5    6    7    8    9
row_0                                                 
0      327   19  73   56   42   10   19   35   56   30
1        6  169   7   21   26   16    7   19   47   56
2        7   13  95   75   50   13   43   32   26   16
3        6   25  27  158   36   36   44   26   23    6
4       40   26  52   65  113   73   54   62   32   27
5        9   11  14   60   37  124   19   48   22    5
6        5   38  74  130   50   46  159   57   36    9
7       23   11  46   68   51   78   96  167   11   12
8       65   15  14   24   25    9    6    4  209   21
9       45  121   8   33   49   11   29   47   64  343
p-value from mcnemar test: 0.01463329792022705


Both wilconxon and McNemar returned *p-values* bellow 5%, indicating model **a** is indeed better than model **b**. Wilconxon, however, is more powerful (0.00769 < 0.01463).